# Exemple de base AutoGen

Dans cet exemple de code, vous utiliserez le cadre d'IA [AutoGen](https://aka.ms/ai-agents/autogen) pour créer un agent de base.

L'objectif de cet exemple est de vous montrer les étapes que nous utiliserons plus tard dans les exemples de code supplémentaires lors de la mise en œuvre des différents modèles d'agents.


## Importer les bibliothèques Python nécessaires


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Créer le Client

Dans cet exemple, nous utiliserons [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) pour accéder au LLM.

Le `model` est défini comme `gpt-4o-mini`. Essayez de changer le modèle pour un autre disponible sur le marketplace de GitHub Models afin de voir les différents résultats.

Pour un test rapide, nous allons simplement exécuter une invite simple - `Quelle est la capitale de la France`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Définir l'Agent

Maintenant que nous avons configuré le `client` et confirmé qu'il fonctionne, créons un `AssistantAgent`. Chaque agent peut se voir attribuer :  
**name** - Un nom court qui sera utile pour le référencer dans des flux multi-agents.  
**model_client** - Le client que vous avez créé à l'étape précédente.  
**tools** - Les outils disponibles que l'Agent peut utiliser pour accomplir une tâche.  
**system_message** - Le méta-prompt qui définit la tâche, le comportement et le ton du LLM.  

Vous pouvez modifier le message système pour voir comment le LLM réagit. Nous aborderons les `tools` dans la Leçon n°4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Exécuter l'Agent

La fonction ci-dessous permettra de lancer l'agent. Nous utilisons la méthode `on_message` pour mettre à jour l'état de l'agent avec le nouveau message.

Dans ce cas, nous mettons à jour l'état avec un nouveau message de l'utilisateur qui est : `"Planifie-moi des vacances ensoleillées géniales"`.

Vous pouvez modifier le contenu du message pour voir comment le LLM répond différemment.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
